In [51]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm
import atlite

The objective of this code is to use the PV model we selected to obtain a long-term PV CF time series that we can store and later use to run our analysis.

## Necessary steps:

Step 1: Load the ERA5 data and create a cutout

Step 2: Load the farm data

Step 3: Run the atlite PV model for the entire period

Step 3: Save the data into a file to use later in the analysis

### Step 1: Load the ERA5 data and create a cutout

Load the ERA5 data

In [ ]:
ds_temperature_2018_2023 = xr.open_dataset('../data/ERA5/ERA5_2m_temperature_soil_temperature_level_4_hourly_2018_2023.nc')
ds_radiation_sfc_2018_2023 = xr.open_dataset('../data/ERA5/ERA5_surface_net_solar_radiation_surface_solar_radiation_downwards_hourly_2018_2023.nc')
ds_radiation_toa_2018_2023 = xr.open_dataset('../data/ERA5/ERA5_toa_incident_solar_radiation_total_sky_direct_solar_radiation_at_surface_hourly_2018_2023.nc')

ds_temperature_2018_2023 = ds_temperature_2018_2023.reduce(np.nanmean, dim='expver',keep_attrs=True)
ds_radiation_sfc_2018_2023 = ds_radiation_sfc_2018_2023.reduce(np.nanmean, dim='expver',keep_attrs=True)
ds_radiation_toa_2018_2023 = ds_radiation_toa_2018_2023.reduce(np.nanmean, dim='expver',keep_attrs=True)

In [ ]:
ds_temperature_1988_2017 = xr.open_mfdataset(['../data/ERA5/ERA5_2m_temperature_soil_temperature_level_4_hourly_{}_{}.nc'.format(yy, yy+5) for yy in np.arange(1988, 2013, 6, dtype=int)])
ds_radiation_sfc_1988_2017 = xr.open_mfdataset(['../data/ERA5/ERA5_surface_net_solar_radiation_surface_solar_radiation_downwards_hourly_{}_{}.nc'.format(yy, yy+5) for yy in np.arange(1988, 2013, 6, dtype=int)])
ds_radiation_toa_1988_2017 = xr.open_mfdataset(['../data/ERA5/ERA5_toa_incident_solar_radiation_total_sky_direct_solar_radiation_at_surface_hourly_{}_{}.nc'.format(yy, yy+5) for yy in np.arange(1988, 2013, 6, dtype=int)])

In [ ]:
list_files_era5_old = []
filepath_base = '../data/ERA5/ERA5_all_variables_{}_{:02d}.nc'
for year in np.arange(1940,1988, dtype=int):
    for month in np.arange(1,13, dtype=int):
        list_files_era5_old.append(filepath_base.format(year, month))

In [55]:
ds_1940_1987 = xr.open_mfdataset(list_files_era5_old)

In [56]:
ds_1940_1987 = ds_1940_1987[['tisr', 'fdir', 'ssr', 'ssrd', 't2m', 'stl4']]

In [57]:
ds = xr.concat([ds_1940_1987,
                xr.merge([ds_temperature_1988_2017, ds_radiation_sfc_1988_2017, ds_radiation_toa_1988_2017]),
                xr.merge([ds_temperature_2018_2023, ds_radiation_sfc_2018_2023, ds_radiation_toa_2018_2023])
                ], dim='time')

In [58]:
cutout_ie = atlite.cutout.get_cutout_from_era5_data('path', ds, ['influx', 'temperature'])

### Step 2: Load the farm data

In [ ]:
# Read csv file located in /Data folder
df_capacity_solar = pd.read_csv('../data/202406_solar_capacity_eir_manualfarms.csv', index_col=0)
# Rename the columns to fit Atlite
df_capacity_solar = df_capacity_solar.dropna().rename(columns={'Capacity (MW)':'MEC', 'latitude':'y', 'longitude':'x'})
df_capacity_solar = df_capacity_solar[df_capacity_solar['Connection date'] < '2024']
df_capacity_solar['Connection date'] = pd.to_datetime(df_capacity_solar['Connection date'])
df_capacity_solar['capacity'] = df_capacity_solar['MEC']*1.4

We change the connection date for the large capacities installed in 2023 whose installation date we estimated from the EirGrid data and simulations using this capacity. Namely:

- First two IC installations: 2023-04-01 -> "Rosspile" (95 MW) and "Gillinstown" (95 MW)
- Third IC installation: 2023-04-26 -> "Gallanstown" (119 MW)
- Fourth IC installation: 2023-06-12 -> "Blundelstown" (60 MW)

In [60]:
# df_capacity_solar.loc['Rosspile', 'Connection date'] = pd.to_datetime('2023-04-01')
# df_capacity_solar.loc['Gillinstown', 'Connection date'] = pd.to_datetime('2023-04-01')
# df_capacity_solar.loc['Gallanstown', 'Connection date'] = pd.to_datetime('2023-04-26')
# df_capacity_solar.loc['Blundelstown', 'Connection date'] = pd.to_datetime('2023-06-12')

In [61]:
df_capacity_solar

,MEC,Connection date,Region,ROI/NI,x,y,capacity
Name,,,,,,,
Hortland PV,14.000,2022-07-01,Dunfirth,ROI,-6.784832,53.382133,19.6000
Hilltown PV,10.000,2023-04-27,BALTRASNA,ROI,-6.433524,53.579615,14.0000
Millvale North PV,8.000,2022-04-25,Ballybeg,ROI,-6.107515,52.995926,11.2000
Davidstown Solar,5.500,2022-10-31,Wexford,ROI,-6.628588,52.471481,7.7000
Knockglass Solar Farm (Bawnmore extension),4.000,2022-11-21,Macroom,ROI,-8.914000,51.953417,5.6000
Curraghmartin Solar Park,3.990,2022-07-26,Waterford,ROI,-7.216795,52.282362,5.5860
Lurrig Solar Farm,4.200,2022-12-14,Midleton,ROI,-8.160190,51.842968,5.8800
Lurrig Solar Ext.,0.340,2023-07-21,Midleton,ROI,-8.160190,51.842968,0.4760
Connacht GAA Solar PV,0.300,2023-03-30,Dalton,ROI,-8.852849,53.774640,0.4200


### Step 3: Run atlite

In [62]:
def get_cf_series_atlite(cutout_ie: atlite.Cutout, df_capacity: pd.DataFrame, panel_model: str, only_cf: bool = True):
    summed_time_series = np.zeros(cutout_ie.data['time'].shape)
    total_capacity_time_series  = np.zeros(cutout_ie.data['time'].shape)

    # Iterate over all capacity installations
    for idx, row in tqdm(df_capacity.reset_index().iterrows(), total= df_capacity.shape[0]):
        cap = row['capacity']
        mec = row['MEC']
        df_capacity_i = pd.DataFrame([row])
        layout = cutout_ie.layout_from_capacity_list(df_capacity_i, col="capacity")

        time_series = cutout_ie.pv(layout=layout,
                                    panel=atlite.convert.get_solarpanelconfig(panel_model),
                                    orientation=atlite.convert.get_orientation('latitude_optimal'),
                                    tracking=None)

        time_series = time_series.squeeze()
        time_series[time_series>mec] = mec

        capacity_time_series = time_series.copy()
        capacity_time_series[:] = mec

    # Add the mean CF time series to the total multiplied by the capacity (weight)
        summed_time_series += time_series[:]
        total_capacity_time_series += capacity_time_series[:]

    # Divide the total time series by the total IC to go back to CF
    cf_time_series = summed_time_series/total_capacity_time_series
    if only_cf:
        return cf_time_series
    return cf_time_series, summed_time_series, total_capacity_time_series

In [63]:
pv_cf, pv_gen, pv_ic = get_cf_series_atlite(cutout_ie=cutout_ie, df_capacity=df_capacity_solar, panel_model='KANENA', only_cf=False)

  0%|          | 0/49 [00:00<?, ?it/s]

[#######                                 ] | 19% Completed | 6.79 s

/Users/aina/anaconda3/envs/atlite/lib/python3.11/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in log
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 23.35 s


  2%|▏         | 1/49 [00:25<20:18, 25.38s/it]

...

 98%|█████████▊| 48/49 [27:19<00:30, 30.37s/it]

[#######                                 ] | 19% Completed | 9.53 s

/Users/aina/anaconda3/envs/atlite/lib/python3.11/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in log
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 34.27 s


100%|██████████| 49/49 [27:56<00:00, 34.22s/it]


In [65]:
pv_cf.to_netcdf('../data/pv_cf_1940_2023.nc')